# README

#### 放置数据
将 本笔记本 和 p2rank_2.3 文件夹置于 TankBind/example 下

在 TankBind/example 下建立文件夹 inputs 文件夹

在 input 文件夹中放入上一格提到的 .csv 文件和蛋白质对应的 .pdb 文件（同样的蛋白质文件/csv文件只用放一次）

- .pdb 文件：名字格式为：`f"{pdb_name}".pdb`，此`pdb_name`参数在下文填充。例如："7kac.pdb"

- .csv 文件：列名为`id`和`smiles`，`id` 用于区分不同的 SMILES，可为任意值）

#### 小分子构象文件
生成的小分子构象文件中，若文件名
- 包含 `_AfGv_` 字段，说明手动指定的口袋和由 TankBind 依据最高 Affinity 选择的口袋为同一口袋

否则，将会分别生成：
- 包含中间字段 `_Af_` 的构象：由 TankBind 选择的口袋
- 包含中间字段 `_Gv_` 的构象：由`given_pocket`参数指定的口袋

# Input


`conf_by_chosen` 参数：若为 `True`，则 TankBind 会依据预测出的最高 Affinity 选择对应口袋并生成构象。

`distinguish_by_timestamp` 参数：若为 `True`，则 输出文件夹名字会额外包含一个时间戳后缀。

填完下边第一个框就可以直接跑了

In [1]:
pdb_name = "7kac"
cp_name = "HPK14" # Please remove the "_"
csv_name = "HPK14-ID-SMILES.csv"  # Titles: "id", "smiles"

given_pocket = 0
given_pocket = "pocket_1" # or 0

base_pre = f"./PREDICTION"

conf_by_chosen = True
# if false and "given_pocket" is 0, only the conformations relating to the given_pocket will be generated.

distinguish_by_timestamp = True 
# if true, the output folder name will be suffiexed with a timestamp 

In [2]:
if (given_pocket == 0) and (conf_by_chosen == False):
    conf_by_chosen = True

tankbind_src_folder_path = "../tankbind/"
import sys
sys.path.insert(0, tankbind_src_folder_path)
from Bio.PDB.PDBList import PDBList   # pip install biopython if import failure
import os
import numpy as np
import pandas as pd
import time

In [3]:
if distinguish_by_timestamp:
    timetag = time.strftime("%m%d%H%M")
    pre = f"{base_pre}/{cp_name}-{pdb_name}-{timetag}"
else:
    pre = f"{base_pre}/{cp_name}-{pdb_name}"
pdir = f"{pre}/PDBs/"
os.system(f"mkdir -p {base_pre}")
os.system(f"rm -rf {pre}/sdfs")
os.system(f"mkdir -p {pre}/sdfs")
os.system(f"rm -rf {pre}/PDBs")
os.system(f"mkdir -p {pre}/PDBs")
os.system(f"rm -rf {pre}/p2rank")
os.system(f"mkdir -p {pre}/p2rank")
os.system(f"cp ./inputs/{csv_name} {pre}")
os.system(f"cp ./inputs/{pdb_name}.pdb {pre}")

0

In [4]:
pdb = f'{pdb_name}'
os.system(f"mkdir -p {pdir}")
pdbl = PDBList()
native_pdb = f"{pre}/{pdb_name}.pdb"

from Bio.PDB import PDBParser
parser = PDBParser(QUIET=True)
s = parser.get_structure(pdb, native_pdb)



In [5]:
proteinFile = f"{pre}/{pdb}.pdb"

In [6]:
import rdkit.Chem as Chem
from feature_utils import generate_sdf_from_smiles_using_rdkit

In [7]:
input_dataframe = pd.read_csv(f'{pre}/{csv_name}')
smilesdict = dict(zip(input_dataframe['id'], input_dataframe['smiles']))
for _key in smilesdict.keys():
    rdkitMolFile = f"{pre}/sdfs/{pdb}_{_key}_mol_from_rdkit.sdf"
    shift_dis = 0   # for visual only, could be any number, shift the ligand away from the protein.
    generate_sdf_from_smiles_using_rdkit(smilesdict[_key], rdkitMolFile, shift_dis=shift_dis)

# get protein feature

In [8]:
from feature_utils import get_protein_feature

In [9]:
parser = PDBParser(QUIET=True)
s = parser.get_structure("x", proteinFile)
res_list = list(s.get_residues())

In [10]:
protein_dict = {}
protein_dict[pdb] = get_protein_feature(res_list)


# get compound feature

In [11]:
from feature_utils import extract_torchdrug_feature_from_mol

In [12]:
compound_dict = {}
for ligandName in smilesdict.keys():
    rdkitMolFile = f"{pre}/sdfs/{pdb}_{ligandName}_mol_from_rdkit.sdf"
    mol = Chem.MolFromMolFile(rdkitMolFile)
    compound_dict[pdb+f"_{ligandName}"+"_rdkit"] = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)

# p2rank

In [13]:
pdb_list = [pdb]
ds = f"{pre}/protein_list.ds"
with open(ds, "w") as out:
    for pdb in pdb_list:
        out.write(f"./{pdb}.pdb\n")


In [14]:
p2rank = "bash ./p2rank_2.3/prank"
cmd = f"{p2rank} predict {ds} -o {pre}/p2rank -threads 1"
os.system(cmd)

----------------------------------------------------------------------------------------------
 P2Rank 2.3
----------------------------------------------------------------------------------------------

predicting pockets for proteins from dataset [protein_list.ds]
processing [7kac.pdb] (1/1)
predicting pockets finished in 0 hours 0 minutes 10.391 seconds
results saved to directory [/home/jovyan/TankBind/examples/PREDICTION/HPK14-7kac-07220904/p2rank]

----------------------------------------------------------------------------------------------
 finished successfully in 0 hours 0 minutes 11.182 seconds
----------------------------------------------------------------------------------------------


0

In [15]:
info = []
for pdb in pdb_list:
    for compound_name in list(compound_dict.keys()):
        # use protein center as the block center.
        com = ",".join([str(a.round(3)) for a in protein_dict[pdb][0].mean(axis=0).numpy()])
        info.append([pdb, compound_name, "protein_center", com])

        p2rankFile = f"{pre}/p2rank/{pdb}.pdb_predictions.csv"
        pocket = pd.read_csv(p2rankFile)
        pocket.columns = pocket.columns.str.strip()
        pocket_coms = pocket[['center_x', 'center_y', 'center_z']].values
        for ith_pocket, com in enumerate(pocket_coms):
            com = ",".join([str(a.round(3)) for a in com])
            info.append([pdb, compound_name, f"pocket_{ith_pocket+1}", com])
info = pd.DataFrame(info, columns=['protein_name', 'compound_name', 'pocket_name', 'pocket_com'])
info

,protein_name,compound_name,pocket_name,pocket_com
0,7kac,7kac_HPK14-001_rdkit,protein_center,"22.207,-63.269,-21.249"
1,7kac,7kac_HPK14-001_rdkit,pocket_1,"4.859,-57.458,-43.996"
2,7kac,7kac_HPK14-001_rdkit,pocket_2,"27.753,-82.781,-1.496"
3,7kac,7kac_HPK14-001_rdkit,pocket_3,"20.086,-94.025,-5.626"
4,7kac,7kac_HPK14-001_rdkit,pocket_4,"21.08,-56.429,-46.002"
...,...,...,...,...
947,7kac,7kac_HPK14-068_rdkit,pocket_9,"27.432,-83.485,-11.664"
948,7kac,7kac_HPK14-068_rdkit,pocket_10,"20.464,-61.454,-27.098"
949,7kac,7kac_HPK14-068_rdkit,pocket_11,"28.164,-56.3,-21.221"
950,7kac,7kac_HPK14-068_rdkit,pocket_12,"34.073,-94.837,-5.136"


In [16]:
p2rankFile = "PREDICTION/HPK14-7kac/p2rank/7kac.pdb_predictions.csv"
pocket = pd.read_csv(p2rankFile)

# construct dataset

In [17]:
import torch
torch.set_num_threads(1)

In [18]:
from data import TankBind_prediction

In [19]:
dataset_path = f"{pre}/{pdb}_dataset/"
os.system(f"rm -r {dataset_path}")
os.system(f"mkdir -p {dataset_path}")
dataset = TankBind_prediction(dataset_path, data=info, protein_dict=protein_dict, compound_dict=compound_dict)

rm: cannot remove './PREDICTION/HPK14-7kac-07220904/7kac_dataset/': No such file or directory
Processing...
Done!


['PREDICTION/HPK14-7kac-07220904/7kac_dataset/processed/data.pt', 'PREDICTION/HPK14-7kac-07220904/7kac_dataset/processed/protein.pt', 'PREDICTION/HPK14-7kac-07220904/7kac_dataset/processed/compound.pt']


In [20]:
dataset_path = f"{pre}/{pdb}_dataset/"
dataset = TankBind_prediction(dataset_path)

['PREDICTION/HPK14-7kac-07220904/7kac_dataset/processed/data.pt', 'PREDICTION/HPK14-7kac-07220904/7kac_dataset/processed/protein.pt', 'PREDICTION/HPK14-7kac-07220904/7kac_dataset/processed/compound.pt']


In [21]:
import logging
from torch_geometric.loader import DataLoader
from tqdm import tqdm    # pip install tqdm if fails.
from model import get_model
# from utils import *

In [22]:
batch_size = 5
device = 'cuda' if torch.cuda.is_available() else 'cpu'
logging.basicConfig(level=logging.INFO)
model = get_model(0, logging, device)
# re-dock model
# modelFile = "../saved_models/re_dock.pt"
# self-dock model
modelFile = "../saved_models/self_dock.pt"

model.load_state_dict(torch.load(modelFile, map_location=device))
_ = model.eval()

data_loader = DataLoader(dataset, batch_size=batch_size, follow_batch=['x', 'y', 'compound_pair'], shuffle=False, num_workers=8)
affinity_pred_list = []
y_pred_list = []
for data in tqdm(data_loader):
    data = data.to(device)
    y_pred, affinity_pred = model(data)
    affinity_pred_list.append(affinity_pred.detach().cpu())
    for i in range(data.y_batch.max() + 1):
        y_pred_list.append((y_pred[data['y_batch'] == i]).detach().cpu())

affinity_pred_list = torch.cat(affinity_pred_list)

09:05:04   5 stack, readout2, pred dis map add self attention and GVP embed, compound model GIN
Parameter containing:
tensor([1.], requires_grad=True)


100%|██████████| 191/191 [00:12<00:00, 15.14it/s]


In [23]:
info = dataset.data
info['affinity'] = affinity_pred_list

In [24]:
info.to_csv(f"{pre}/result_info_all.csv")

In [25]:
chosen = info.loc[info.groupby(['protein_name', 'compound_name'],sort=False)['affinity'].agg('idxmax')].reset_index()
chosen.to_csv(f"{pre}/result_chosen_by_affinity.csv")

In [26]:
if given_pocket:
    info_right_pocket = info[info["pocket_name"]==given_pocket].reset_index()
    info_right_pocket.to_csv(f"{pre}/result_given_pocket.csv")

# from predicted interaction distance map to sdf

In [27]:
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
from generation_utils import get_LAS_distance_constraint_mask, get_info_pred_distance, write_with_new_coords
device = 'cpu'
if conf_by_chosen:
    for i, line in chosen.iterrows():
        idx = line['index']
        pocket_name = line['pocket_name']
        compound_name = line['compound_name']
        ligandName = compound_name.split("_")[1]
        coords = dataset[idx].coords.to(device)
        protein_nodes_xyz = dataset[idx].node_xyz.to(device)
        n_compound = coords.shape[0]
        n_protein = protein_nodes_xyz.shape[0]
        y_pred = y_pred_list[idx].reshape(n_protein, n_compound).to(device)
        y = dataset[idx].dis_map.reshape(n_protein, n_compound).to(device)
        compound_pair_dis_constraint = torch.cdist(coords, coords)
        rdkitMolFile = f"{pre}/sdfs/{pdb}_{ligandName}_mol_from_rdkit.sdf"
        mol = Chem.MolFromMolFile(rdkitMolFile)
        LAS_distance_constraint_mask = get_LAS_distance_constraint_mask(mol).bool()
        info2 = get_info_pred_distance(coords, y_pred, protein_nodes_xyz, compound_pair_dis_constraint, 
                                      LAS_distance_constraint_mask=LAS_distance_constraint_mask,
                                      n_repeat=1, show_progress=False)

        result_folder = f'{pre}/{pdb}_result/'
        os.system(f'mkdir -p {result_folder}')
        # toFile = f'{result_folder}/{ligandName}_{pocket_name}_tankbind.sdf'
        if pocket_name == given_pocket:
            toFile = f'{result_folder}/{ligandName}_tankbind_AfGv_{pocket_name}.sdf'
        else:
            toFile = f'{result_folder}/{ligandName}_tankbind_Af_{pocket_name}.sdf'
        # print(toFile)
        new_coords = info2.sort_values("loss")['coords'].iloc[0].astype(np.double)
        write_with_new_coords(mol, new_coords, toFile)

if given_pocket:
    for i, line in info_right_pocket.iterrows():
        idx = line['index']
        pocket_name = line['pocket_name']
        if (pocket_name == given_pocket) and conf_by_chosen:
            pass
        else:
            compound_name = line['compound_name']
            ligandName = compound_name.split("_")[1]
            coords = dataset[idx].coords.to(device)
            protein_nodes_xyz = dataset[idx].node_xyz.to(device)
            n_compound = coords.shape[0]
            n_protein = protein_nodes_xyz.shape[0]
            y_pred = y_pred_list[idx].reshape(n_protein, n_compound).to(device)
            y = dataset[idx].dis_map.reshape(n_protein, n_compound).to(device)
            compound_pair_dis_constraint = torch.cdist(coords, coords)
            rdkitMolFile = f"{pre}/sdfs/{pdb}_{ligandName}_mol_from_rdkit.sdf"
            mol = Chem.MolFromMolFile(rdkitMolFile)
            LAS_distance_constraint_mask = get_LAS_distance_constraint_mask(mol).bool()
            info2 = get_info_pred_distance(coords, y_pred, protein_nodes_xyz, compound_pair_dis_constraint, 
                                          LAS_distance_constraint_mask=LAS_distance_constraint_mask,
                                          n_repeat=1, show_progress=False)

            result_folder = f'{pre}/{pdb}_result/'
            os.system(f'mkdir -p {result_folder}')
            # toFile = f'{result_folder}/{ligandName}_{pocket_name}_tankbind.sdf'
            toFile = f'{result_folder}/{ligandName}_tankbind_Gv_{pocket_name}.sdf'
            # print(toFile)
            new_coords = info2.sort_values("loss")['coords'].iloc[0].astype(np.double)
            write_with_new_coords(mol, new_coords, toFile)

NameError: name 'right_pocket' is not defined